In [1]:
if 'google.colab' in str(get_ipython()):
  print('Running on CoLab')
  %matplotlib inline
else:
    print('Not running on CoLab')
    %matplotlib notebook
    import os
    os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
    os.environ["CUDA_VISIBLE_DEVICES"] = "1"

Not running on CoLab


In [26]:
import numpy as np
import time
import scipy as sp

import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

from keras.models import Sequential
from keras.layers import Dense, GRU, Conv2D, Flatten, MaxPooling2D, Dropout, Reshape, TimeDistributed
from IPython.display import HTML
from funtion_file import matrix_maker
from tensorflow.keras.metrics import Precision, Recall, AUC

class MatrixMaker:
    __slots__ = ("rows", "cols", "kernel_size", "line_size", 'num_per_mat', 'smooth_matrix', 'line_start_position', 'alfa', 'matrix_fade', 'line_pos_mat')
    def __init__(self, rows, cols=None, kernel_size=(1, 1), line_size=(1, 2), num_per_mat=10):
        self.rows = rows
        self.cols = cols or rows
        self.kernel_size = kernel_size
        self.line_size = line_size
        self.num_per_mat = num_per_mat

        self.smooth_matrix = self.create_smoothed_matrix()
        self.line_start_position = self.create_line_start_position()
        self.alfa = self.create_alfa()
        self.matrix_fade = self.create_matrix_line_fade()
        self.line_pos_mat = self.create_line_pos_mat()

    def create_smoothed_matrix(self):
        kernel = np.ones(shape=self.kernel_size, dtype=float) / (self.kernel_size[0] * self.kernel_size[1])
        return sp.ndimage.convolve(np.random.rand(self.rows, self.cols), kernel)

    def create_line_start_position(self):
        return (np.random.randint(low=0, high=self.rows - self.line_size[0] + 1),
                np.random.randint(low=0, high=self.cols - self.line_size[1] + 1))

    def create_alfa(self):
        return np.linspace(1, 0, num=self.num_per_mat)

    def create_matrix_with_line(self, alfa):
        matrix = np.ones((self.rows, self.cols))
        matrix[self.line_start_position[0]:self.line_start_position[0] + self.line_size[0],
        self.line_start_position[1]:self.line_start_position[1] + self.line_size[1]] = alfa
        return matrix

    def create_matrix_line_fade(self):
        matrix_line_fade = []
        for i in range(self.num_per_mat):
            line = self.create_matrix_with_line(self.alfa[i])
            matrix_line_fade.append(self.smooth_matrix * line)

        return matrix_line_fade

    def create_line_pos_mat(self):
        return np.array(np.logical_not(self.create_matrix_with_line(0)).astype(int), dtype='float16')


class MatrixLister:
    def __init__(self, row_len, col_len, kernel_size, min_max_line_size, rotate, num_of_mat, num_per_mat, num_neuron):
        self.row_len = row_len
        self.col_len = col_len
        self.kernel_size = kernel_size
        self.min_max_line_size = min_max_line_size
        self.rotate = rotate
        self.num_of_mat = num_of_mat
        self.num_per_mat = num_per_mat
        self.num_neuron = num_of_neurons

        self.matrix_list = self.create_matrix_in_list()
        self.con_matrix, self.con_alfa = self.concatenate_matrices()

        self.neural_network = NeuralNetwork(input_size=(row_len, col_len), num_neuron=self.num_neuron)

    def create_matrix_in_list(self):
        line_sizes = [rotater((np.random.randint(self.min_max_line_size[0][0], self.min_max_line_size[1][0] + 1),
                       np.random.randint(self.min_max_line_size[0][1], self.min_max_line_size[1][1] + 1)))
                      for _ in range(self.num_of_mat)]

        return [MatrixMaker(self.row_len, self.col_len, self.kernel_size, line_sizes[i], self.num_per_mat)
                for i in range(self.num_of_mat)]

    def concatenate_matrices(self):
        concatenated_matrices = []
        con_alfa = []

        for matrix in self.matrix_list:
            concatenated_matrices += matrix.create_matrix_line_fade()
            con_alfa += list(matrix.alfa)

        return np.array(concatenated_matrices, dtype='float32'), con_alfa

    def con_line_pos_mat(self):
        con_line_pos_mat = []
        for matrix in self.matrix_list:
            for alfa in matrix.alfa:
                if alfa != 1:
                    con_line_pos_mat.append(matrix.line_pos_mat)
                else:
                    con_line_pos_mat.append(np.zeros((self.row_len, self.col_len)))
                    
                    
        fisk = np.stack(con_line_pos_mat)
        return fisk
    
    def make_new_matrix_in_list(self, **kwargs):
        update = False
        for key, value in kwargs.items():
            if key in ['row_len', 'col_len']:
                update = True
            if hasattr(self, key):
                setattr(self, key, value)
            else:
                print(f"Attribute {key} does not exist in the class.")
        
        self.matrix_list = self.create_matrix_in_list()
        self.con_matrix, self.con_alfa = self.concatenate_matrices()
        if update:
            self.neural_network = NeuralNetwork(input_size=(row_len, col_len), num_neuron=self.num_neuron)

    def train_neural_network(self, num_epochs=10, batch_size=64):

        self.neural_network.train(self.con_matrix, self.con_line_pos_mat(), num_epochs, batch_size)

    def plot(self, interval=200):
        fig, ax = plt.subplots()

        def update(frame):
            ax.clear()
            im = ax.imshow(self.con_matrix[frame], interpolation='nearest', aspect='auto', vmin=0, vmax=1)

            return [im]

        animation = FuncAnimation(fig, update, frames=len(self.con_matrix), interval=interval, repeat=False, blit=True)
        plt.tight_layout()
        plt.show(block=False)
        plt.show()
        return animation
    
    def plot_matrices(self, num_to_pred, new_mat=False, interval=500):
        if new_mat:
            self.num_of_mat = num_to_pred
            self.matrix_list = self.create_matrix_in_list()
            self.con_matrix, self.con_alfa = self.concatenate_matrices()
            
        input_matrix = np.array(self.con_matrix[:num_to_pred*self.num_per_mat])
        true_matrix = np.array(self.con_line_pos_mat())
        pred = self.neural_network.predict(input_matrix)
    
        predicted_line_pos_mat = np.array(pred).reshape(input_matrix.shape)
        
        fig, axes = plt.subplots(1, 3, figsize=(12, 4))  # 1 row, 3 columns
    
        def update(frame):
            # Plot Input Matrix
            im = [axes[0].imshow(input_matrix[frame], interpolation='nearest', aspect='auto', vmin=0, vmax=1)]
            axes[0].set_title('Input Matrix')
    
            # Plot True Line Position Matrix
            im.append(axes[1].imshow(true_matrix[frame], interpolation='nearest', aspect='auto', vmin=0, vmax=1))
            axes[1].set_title('True Line Position Matrix')
    
            # Plot Predicted Line Position Matrix
            im.append(axes[2].imshow(predicted_line_pos_mat[frame], interpolation='nearest', aspect='auto', vmin=0, vmax=1))
            axes[2].set_title('Predicted Line Position Matrix')
            
            return im
    
        animation = FuncAnimation(fig, update, frames=len(input_matrix), interval=interval, repeat=False, blit=True)
    
        plt.tight_layout()
        plt.show(block=False)
        return animation


def rotater(line):
    if np.random.random() < 0.5:
        return line[::-1]
    return line


class NeuralNetwork:
    def __init__(self, input_size, num_neuron):
        self.model = self.build_model(input_size, num_neuron)

    def build_model(self, input_size, num_neuron):
        model = Sequential()
        
        # Apply Conv2D and Flatten to each time step
        model.add(Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu'))
        model.add(Conv2D(64, kernel_size=(2, 2), padding='same', activation='relu'))
        model.add(MaxPooling2D(pool_size=2))
        model.add(Conv2D(128, kernel_size=(2, 2), padding='same', activation='relu'))
        model.add(MaxPooling2D(pool_size=2))
        # Apply SimpleRNN to the output of Conv2D and Flatten
        model.add(TimeDistributed(Flatten()))
        model.add(GRU(num_neuron, activation='tanh', return_sequences=True))
        model.add(Dropout(0.1))
        model.add(GRU(num_neuron, activation='tanh'))
        
        # Fully connected layer
        model.add(Dense(input_size[0] * input_size[1], activation='sigmoid'))
        
        # Reshape to the desired output shape
        model.add(Reshape((input_size[0], input_size[1], 1)))
        
        model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=[
            'accuracy',
            Precision(name='precision'),
            Recall(name='recall')]
        )
        return model

    def train(self, input_data, output_data, epochs, batch_size):
        in_data = np.expand_dims(input_data, -1)
        out_data = np.expand_dims(output_data, -1)

        self.model.fit(in_data, out_data, epochs=epochs, batch_size=batch_size)

    def predict(self, input_matrices):
        # Flatten and concatenate all input matrices in the batch
        input_data = np.expand_dims(np.array([matrix for matrix in input_matrices]),-1)
    
        # Predict the output for the entire batch
        predicted_output = self.model.predict(input_data)
    
        return predicted_output




    

In [27]:
row_len = 40
col_len = 40
kernel_size = (3, 3)
min_max_line_size = [(1,4),(2,5)]
rotate = True
num_of_mat = 100
numb_of_picture = 8
num_of_neurons = 256

matrix_lister = MatrixLister(row_len, col_len, kernel_size, min_max_line_size, rotate, num_of_mat, numb_of_picture, num_of_neurons)

In [28]:
batch_size = 2**6
start_epochs = 50
end_epochs = 50
hyper_epochs = 2

epochs_list = [round((-start_epochs+end_epochs)/(1+np.exp(-4*(x-hyper_epochs/5)))+start_epochs) for x in range(hyper_epochs)]

start = time.time()
for epochs in epochs_list:
    matrix_lister.train_neural_network(batch_size=batch_size, num_epochs=epochs)
    matrix_lister.make_new_matrix_in_list(kernel_size=(2,2), num_of_mat=500)
print(time.time() - start)

Epoch 1/50
4/4 [==============================] - 3s 122ms/step - loss: 0.6844 - accuracy: 0.6500 - precision: 0.0037 - recall: 0.3493
Epoch 2/50
4/4 [==============================] - 1s 117ms/step - loss: 0.4599 - accuracy: 0.9529 - precision: 0.0044 - recall: 0.0522
Epoch 3/50
4/4 [==============================] - 1s 121ms/step - loss: 0.1811 - accuracy: 0.9963 - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 4/50
4/4 [==============================] - 1s 123ms/step - loss: 0.0813 - accuracy: 0.9963 - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 5/50
4/4 [==============================] - 1s 127ms/step - loss: 0.0449 - accuracy: 0.9963 - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 6/50
4/4 [==============================] - 1s 126ms/step - loss: 0.0319 - accuracy: 0.9963 - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 7/50
4/4 [==============================] - 1s 119ms/step - loss: 0.0270 - accuracy: 0.9963 - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 8/50
4/4 

In [25]:
ani = matrix_lister.plot_matrices(20)

5/5 [==============================] - 0s 3ms/step


<IPython.core.display.Javascript object>

In [ ]:
if 'google.colab' in str(get_ipython()):
    HTML(ani.to_html5_video())